In [ ]:
import os
from glob import glob
from tqdm.auto import tqdm

from scripts.Colocalization import get_CoLoc

# 2. Setup paths
main_dir = "../../Top1g_NPM1r_UBFcy5_DAPIb"
# This MUST match the output_dir used in your StarDist script
stardist_results_dir = "../../ISD_with_UBF_segmentation_w_STARDIST"
coloc_output_dir = "coloc_test_script2"
# 3. Get experimental groups
groups = [d for d in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, d))]

for g in tqdm(groups, desc="Colocalization Groups"):
    group_path = os.path.join(main_dir, g)
    nd2_files = glob(os.path.join(group_path, "*.nd2"))
    
    for n in tqdm(nd2_files, desc=f"Processing {g}", leave=False):
        # Extract the filename without extension to find the previous results
        basenumber = os.path.splitext(os.path.basename(n))[0]
        
        # Construct paths to the labels generated by the previous StarDist script
        # Note: We use .png as your get_CoLoc script expects skio.imread on them
        nuclei_path = os.path.join(stardist_results_dir, f"{g}_{basenumber}", "nuclei_labels.png")
        npm1_path   = os.path.join(stardist_results_dir, f"{g}_{basenumber}", "nucleoli_labels.png")
        
        # Only run if the segmentation files actually exist
        if os.path.exists(nuclei_path) and os.path.exists(npm1_path):
            try:
                get_CoLoc(
                    nd2_file=n,
                    nucleolar_component="FBL",
                    nuclei_labels_path=nuclei_path,
                    npm1_labels_path=npm1_path,
                    output_dir=coloc_output_dir,
                    group_name=g,
                    NC_IDX=3  # Ensure this matches your FBL channel index
                )
            except Exception as e:
                print(f"Error in colocalization for {basenumber}: {e}")
        else:
            print(f"Skipping {basenumber}: Segmentation labels not found at {nuclei_path}")

print("Colocalization analysis complete!")